In [1]:
from base_layer_utils import BaseLayerDataRepo, BaseLayerResultsRepo, ModelName
from base_layer_utils import LightgbmBLE, SklearnBLE, XGBoostBLE

#from fast_text_data import fasttext_data_process
#from tfidf_data import tfidf_data_process

import pandas as pd
import numpy as np
#from sklearn.cross_validation import KFold # replace with model_selection?
#from sklearn.model_selection import KFold
import time, re, gc
import xgboost as xgb
from sklearn.metrics import roc_auc_score

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
PATH = '~/data/toxic/data/'

train = pd.read_csv(PATH + 'cleaned_train.csv')
test = pd.read_csv(PATH + 'cleaned_test.csv')

print(train.shape)
print(test.shape)

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

(159571, 27)
(153164, 21)


# select layer2 outputs

In [3]:
base_layer_results_repo = BaseLayerResultsRepo()

load from file


In [78]:
_ = base_layer_results_repo.show_scores()

0.9827	ModelName.NBLSVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9826	ModelName.NBSVM_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9819	ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9818	ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9815	ModelName.NBLSVC_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9803	ModelName.NBSVM_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9796	ModelName.LOGREG_wordtfidf_word_(1, 1)_100000_1_1.0_char_(2, 5)_200000_1_1.0
0.9794	ModelName.LGB_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9793	ModelName.LOGREG_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9786	ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w
0.9774	ModelName.NBLSVC_tfidf_word_df2_ng(1, 1)_wmf200000
0.9768	ModelName.NBSVM_tfidf_word_df2_ng(1, 1)_wmf200000
0.9765	ModelName.NBLGB_wordtfidf_word_(1, 1)_100000_1_1.0_char_(2, 5)_200000_1_1.0
0.9765	ModelName.NBLSVC_tf

In [77]:
#base_layer_results_repo.remove('ModelName.NBSVM_tfidf_word_df2_ng(1, 2)_wmf200000')

base_layer_results_repo.remove('ModelName.LGB_layer2')

In [5]:
selected = [
    'ModelName.LGB_w_features_All16_seed1001_layer2',
    'ModelName.LGB_w_features_1stHalf16_seed1001_layer2',
    'ModelName.LOGREG_w_features_All16_seed1001_layer2',
    'ModelName.LOGREG_w_features_2ndHalf16_seed1001_layer2',
    'ModelName.LGB_w_features_2ndHalf16_seed1001_layer2',
    'ModelName.LOGREG_w_features_1stHalf16_seed1001_layer2',
    'ModelName.XGB_w_features_2ndHalf16_seed1001_layer2',
    'ModelName.XGB_w_features_All16_seed1001_layer2',
    'ModelName.XGB_w_features_1stHalf16_seed1001_layer2',
    'ModelName.NBLSVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real',
    'ModelName.LGB_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000'
]

In [79]:
layer1_oof_train_loaded, layer1_oof_test_loaded, layer1_est_preds_loaded = base_layer_results_repo.get_results(threshold=0.976)

In [80]:
for i, key in enumerate(layer1_est_preds_loaded.keys()):
    print(key)
    #print(pd.DataFrame(layer1_est_preds_loaded[key]).describe())
    if i == 0:
        blend = layer1_est_preds_loaded[key]
    else:
        blend += layer1_est_preds_loaded[key]

print(i+1)
blend /= i+1

ModelName.LOGREG_tfidf_word_df2_ng(1, 1)_wmf200000
ModelName.LOGREG_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
ModelName.NBLSVC_tfidf_word_df2_ng(1, 1)_wmf200000
ModelName.NBSVM_tfidf_word_df2_ng(1, 1)_wmf200000
ModelName.LOGREG_wordtfidf_word_(1, 1)_100000_1_1.0_char_(2, 5)_200000_1_1.0
ModelName.NBSVM_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
ModelName.NBSVM_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
ModelName.NBLSVC_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
ModelName.NBLSVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
ModelName.LGB_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
ModelName.NBLGB_wordtfidf_word_(1, 1)_100000_1_1.0_char_(2, 5)_200000_1_1.0
ModelName.NBLSVC_tfidf_word_df2_ng(1, 2)_wmf200000
ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w


In [83]:
pd.DataFrame(blend).describe()

,0,1,2,3,4,5
count,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000
mean,0.238268,0.042403,0.149167,0.032390,0.135564,0.052887
std,0.347316,0.097724,0.285312,0.058814,0.238925,0.107604
min,0.000545,0.002229,0.001659,0.002371,0.001655,0.001755
25%,0.012608,0.006459,0.009911,0.009608,0.011659,0.009109
50%,0.036056,0.009710,0.016530,0.014505,0.023393,0.015000
75%,0.356963,0.021545,0.076996,0.029181,0.113123,0.041675
max,0.999991,0.972263,0.999418,0.989320,0.996741,0.994861


In [99]:
blend.shape

(153164, 6)

In [6]:
layer2_oof_train_loaded, layer2_oof_test_loaded, layer2_est_preds_loaded = base_layer_results_repo.get_results(chosen_ones=selected)

In [7]:
assert len(layer2_oof_train_loaded['toxic']) == len(layer2_est_preds_loaded) == len(selected)

### before we choose which models to assemble, we can do:
#### 1. scatter plot analysis to check the diversity
#### 2. submit to check if the models have similar performance

## Ensembling:

In [8]:
def combine_layer_oof_per_label(layer1_oof_dict, label):
    x = None
    data_list = layer1_oof_dict[label]
    for i in range(len(data_list)):
        if i == 0:
            x = data_list[0]
        else:
            x = np.concatenate((x, data_list[i]), axis=1)
    return x

### 1. simple blend of two models

In [39]:
result = np.empty((test.shape[0],len(label_cols)))

# mix the first two models
for i, label in enumerate(label_cols):
    x_train = combine_layer_oof_per_label(layer2_oof_train_loaded, label)
    x_test = combine_layer_oof_per_label(layer2_oof_test_loaded, label)
    for j in range(x_train.shape[1]):
        roc = roc_auc_score(train[label], x_train[:,j])
        print(label, j, roc) # print out roc for meta feature on meta label (which is just the original train label)
    
    roc_scores_of_a_label = []
    alphas = np.linspace(0,1,1001)
    best_roc = 0
    best_alpha = 0
    for alpha in alphas:
        roc = roc_auc_score(train[label], alpha*x_train[:,0] + (1-alpha)*x_train[:,1])
        if roc > best_roc:
            best_roc = roc
            best_alpha = alpha
    
    print(label, best_roc, best_alpha)
    result[:,i] = best_alpha*x_test[:,0] + (1-best_alpha)*x_test[:,1]

toxic 0 0.9910814282997946
toxic 1 0.990771077509489
toxic 0.9910856078589341 0.9550000000000001
severe_toxic 0 0.9925828223897508
severe_toxic 1 0.9919322096940085
severe_toxic 0.9928764823290488 0.128
obscene 0 0.9962181243081879
obscene 1 0.9959665500533149
obscene 0.9962206810311406 0.984
threat 0 0.9946855970430916
threat 1 0.99310288682231
threat 0.9948137226753532 0.294
insult 0 0.991952202527335
insult 1 0.991528941423655
insult 0.9919546286846996 0.936
identity_hate 0 0.9933174155555204
identity_hate 1 0.9925218214135401
identity_hate 0.9934864460389672 0.203


In [84]:
submission = pd.read_csv(PATH + 'sample_submission.csv')#.head(1000)
submission[label_cols] = result
sub_id = int(time.time())
print(sub_id)
submission.to_csv('./StackPreds/layer2_simpleensemble_' + str(sub_id) + '.csv', index=False)

1521125524


### 2. stacking

In [9]:
len(layer2_oof_train_loaded['toxic'])

11

In [10]:
lgb_stacker_params = {
    'max_depth':3, 
    'metric':"auc", 
    'n_estimators':80, 
    'num_leaves':8, 
    'boosting_type':"gbdt", 
    'learning_rate':0.1, 
    'feature_fraction':0.45,
    'colsample_bytree':0.45, 
    'bagging_fraction':0.8, 
    'bagging_freq':5, 
    'reg_lambda':0.2
}

lgb_stacker = LightgbmBLE(None, None, params=lgb_stacker_params, nb=False, seed=1001)

Naive Bayes is disabled
LightgbmBLE is initialized


In [11]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [ ]:
auc = []
best_rounds = {}

In [25]:
result = np.empty((test.shape[0],len(label_cols)))
for i, label in enumerate(label_cols):
    assert train.shape == (159571, 27)
    x_train = combine_layer_oof_per_label(layer2_oof_train_loaded, label)
    x_test = combine_layer_oof_per_label(layer2_oof_test_loaded, label)
    train_set = lgb.Dataset(x_train, train[label])

    ########## best rounds ##########
    # for cv, val_set is not needed
#     res = lgb.cv(lgb_stacker_params, train_set, seed=1001, 
#            early_stopping_rounds=20, num_boost_round=200)
#     best_rounds[label] = len(res['auc-mean'])
#     auc.append(res['auc-mean'][-1])
#     print('{}:\t\t{:.6f}'.format(label, res['auc-mean'][-1]))
    
    ########## preds ##########
    model = lgb.train(lgb_stacker_params, train_set, num_boost_round=best_rounds[label])
    result[:, i] = model.predict(x_test)
#     lgb_stacker.train(x_train, train[label])
#     result[:, i] = lgb_stacker.predict(x_test)

In [23]:
best_rounds, auc, sum(auc)/6

({'identity_hate': 50,
  'insult': 25,
  'obscene': 76,
  'severe_toxic': 25,
  'threat': 30,
  'toxic': 80},
 [0.9851148841380788,
  0.9901485741396543,
  0.994249389154102,
  0.9809193866673652,
  0.9863534991636291,
  0.9844414726863822],
 0.9868712009915352)

In [26]:
result.shape

(153164, 6)

In [27]:
sub_title = 'layer3_lgb_w_bestrounds_9layer2'
submission = pd.read_csv(PATH + 'sample_submission.csv')
submission[label_cols] = result
tempid = int(time.time())
print(tempid)
submission.to_csv('./StackPreds/TopN_XGB/{}_{}.csv'.format(sub_title, tempid), index=False)

1521085451


In [65]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.923927,0.315186,0.914692,0.193077,0.850242,0.435900
1,0000247867823ef7,0.002651,0.000314,0.000839,0.000295,0.001501,0.000330
2,00013b17ad220c46,0.002769,0.000314,0.000842,0.000295,0.001505,0.000333
3,00017563c3f7919a,0.002645,0.000313,0.000839,0.000295,0.001501,0.000330
4,00017695ad8997eb,0.002689,0.000313,0.000840,0.000295,0.001501,0.000330


In [53]:
from sklearn.linear_model import LogisticRegression
logreg_stacker = SklearnBLE(LogisticRegression, params={}, seed=1001)

In [55]:
model_pool = {}
#model_pool[ModelName.XGB] = xgb_stacker
model_pool[ModelName.LOGREG] = logreg_stacker

In [51]:
from sklearn.model_selection import KFold, StratifiedKFold

def get_oof(clf, x_train, y_train, x_test, nfolds, stratified=False, shuffle=True, seed=1001):
    #pdb.set_trace()
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((nfolds, ntest))
    if stratified:
        kf = StratifiedKFold(n_splits=nfolds, shuffle=shuffle, random_state=seed)
    else:
        kf = KFold(n_splits=nfolds, shuffle=shuffle, random_state=seed)

    for i, (tr_index, te_index) in enumerate(kf.split(x_train, y_train)):
        x_tr, x_te = x_train[tr_index], x_train[te_index]
        y_tr, y_te = y_train.iloc[tr_index], y_train.iloc[te_index]
        
        clf.train(x_tr, y_tr)

        oof_train[te_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)